In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from store import *
import kbech32

In [2]:
store = Store(os.getenv('localappdata') + r'\Kaspad\kaspa-mainnet\datadir2')

In [3]:
store.load_recent_blocks(3600_000)

In [4]:
header_fields = ['timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty']
block_fields = ['pubkey_script', 'kaspad_version', 'miner_version']
count_fields = [] # ['num_parents', 'num_children', 'num_blues', 'num_reds']
frames = store.load_data(header_fields=header_fields, block_fields=block_fields, count_fields=count_fields)

  0%|          | 0/3600 [00:00<?, ?it/s]

In [5]:
frames.keys()

dict_keys(['hash', 'timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty', 'pubkey_script', 'kaspad_version', 'miner_version'])

In [6]:
df = pd.DataFrame(frames).set_index('hash')

In [7]:
pp_time = store.get_header_data(store.pruning_point()).timeInMilliseconds
pp_time, len(df)

(1655592763117, 3600)

In [8]:
df['kaspad_version'].value_counts(normalize=True)

0.12.2    0.566389
0.12.1    0.368056
0.12.0    0.065556
Name: kaspad_version, dtype: float64

In [13]:
def miner_type(miner_version):
    if 'BzMiner' in miner_version:
        return 'BzMiner'
    if '0.2.1-GPU' in miner_version:
        return 'CommunityMiner'
    return miner_version

df["miner_type"] = df["miner_version"].apply(miner_type)

In [14]:
(df['kaspad_version'] + '/' + df['miner_type']).value_counts(normalize=True)

0.12.2/00                0.507778
0.12.1/striker           0.239444
0.12.1/ge.acc-pool.pw    0.066389
0.12.2/BzMiner           0.050833
0.12.1/BzMiner           0.037222
0.12.0/ca.acc-pool.pw    0.031389
0.12.0/ru.acc-pool.pw    0.023333
0.12.1/hk.acc-pool.pw    0.018611
0.12.1/CommunityMiner    0.006389
0.12.0/BzMiner           0.006389
0.12.2/CommunityMiner    0.005556
0.12.0/                  0.003056
0.12.2/                  0.002222
0.12.0/CommunityMiner    0.001389
dtype: float64

In [15]:
df['miner_type'].value_counts(normalize=True)

00                0.507778
striker           0.239444
BzMiner           0.094444
ge.acc-pool.pw    0.066389
ca.acc-pool.pw    0.031389
ru.acc-pool.pw    0.023333
hk.acc-pool.pw    0.018611
CommunityMiner    0.013333
                  0.005278
Name: miner_type, dtype: float64

In [12]:
store.close()